# Feature engineering

## Initialisation

In [ ]:
import numpy as np
import pandas as pd
import statsmodels
import sklearn
import seaborn as sns
import plotly
import plotly.express as px
import matplotlib.pyplot as plt
import itertools
import time
import gc
from tqdm import notebook
from time import sleep

In [ ]:
# Main dataset
# sales = pd.read_csv('../data/sales_train.csv')
sales = pd.read_parquet('../data/output/sales_cleaned.parquet')
# Data-Dicts
items = pd.read_csv('../data/dicts/items.csv')
items_categories = pd.read_csv('../data/dicts/item_categories.csv')
shops = pd.read_csv('../data/dicts/shops.csv')

# Folder - Sumbission data 
submission = pd.read_csv('../data/submission_data/sample_submission.csv')
# a sample submission file in the correct format.

test = pd.read_csv('../data/submission_data/test.csv') 
# the test set. You need to forecast the sales 
# for these shops and products for November 2015.


In [ ]:
sales.head(5)

## full_df - full schema of shops with dicts and target columns adding 

### Creating full schema of montly sold items for every shop - { df } 

In [ ]:
all_obs_combination_by = ['date_block_num', 'shop_id', 'item_id']
all_shops_items = []

for block_num in sales['date_block_num'].unique():
    unique_shops = sales[sales['date_block_num'] == block_num]['shop_id'].unique()
    unique_items = sales[sales['date_block_num'] == block_num]['item_id'].unique()
    all_shops_items.append(np.array(list(itertools.product([block_num], unique_shops, unique_items)), dtype='int32'))

df = pd.DataFrame(np.vstack(all_shops_items), columns=all_obs_combination_by, dtype='int32')
df # full schema with all unique combinations of month number, shop_id, and item_id for month

### Making a target feature and outliers flags from basic dataframe - { aggregated }

In [ ]:
aggregated = sales.groupby(all_obs_combination_by).agg({'item_price'  : 'mean', 'item_cnt_day': 'sum','was_item_price_outlier':'mean', 'was_item_cnt_day_outlier':'mean'})
aggregated.rename(columns={'item_cnt_day': 'item_cnt_month'}, inplace = True)
aggregated[aggregated['was_item_cnt_day_outlier']!=0] # creating additional column for sales as a target

### Merging full schema with agregated by target basic dataframe - contact { full_df } + {test}

In [ ]:
full_df = pd.merge(df, aggregated, on = all_obs_combination_by, how = 'left')
full_df.fillna(value = 0, inplace= True)

test['date_block_num'] = 34
full_df = pd.concat([full_df, test], ignore_index= True)
full_df = full_df.drop('ID',axis=1)

full_df

In [ ]:
(full_df[full_df['item_cnt_month']==0]).shape[0] /  full_df.shape[0]

### Adding dicts on { full_df }

In [ ]:
print(shops.head(5))
print('\n')
print(items.head(1))
print('\n')
print(items_categories.head(1))

In [ ]:
items_categories['general_item_category_name'] = items_categories['item_category_name'].\
    apply(lambda x: 'Игровые консоли' if x.split()[0] == 'Игровые' else x.split()[0] )
items_categories['general_item_category_name'] = pd.Categorical(items_categories.general_item_category_name).codes
items_categories = items_categories.drop('item_category_name', axis=1)

In [ ]:
shops['city'] = shops['shop_name'].apply(lambda x: 'Якутск' if x.split()[0] == '!Якутск' else x.split()[0] )
shops['city'] = pd.Categorical(shops.city).codes
shops = shops.drop('shop_name', axis=1)

In [ ]:
items = items.drop('item_name', axis=1)

In [ ]:
print(shops.head(5))
print('\n')
print(items.head(1))
print('\n')
print(items_categories.head(1))

In [ ]:
full_df = full_df.merge(items, on='item_id', how='left')
full_df = full_df.merge(items_categories, on = 'item_category_id', how = 'left')
full_df = full_df.merge(shops, on = 'shop_id', how = 'left')
full_df # sales DataFrame with full schema of monthly items, aggregated by monthes

We will add dicts data to sales as well for future tranformations

In [ ]:
sales = sales.merge(items, on='item_id', how='left')
sales = sales.merge(items_categories, on = 'item_category_id', how = 'left')
sales = sales.merge(shops, on = 'shop_id', how = 'left')

In [ ]:
size_in_bytes = sales.memory_usage(deep=True).sum()
size_in_megabytes = size_in_bytes / (1024 ** 2)

print(f"Memory usage full_df: {size_in_megabytes:.2f} MB")

In [ ]:
size_in_bytes = full_df.memory_usage(deep=True).sum()
size_in_megabytes = size_in_bytes / (1024 ** 2)

print(f"Memory usage full_df: {size_in_megabytes:.2f} MB")

## Item_cnt_day aggregation based on other features

Aggregating target not only for unique combinations, but also in generall for shops, item_id, category, general_category, and city 

### Sum

We will creater features according to our plan:

In [ ]:
%%time
# all_obs_combination_by

temp = sales.groupby(by=['date_block_num','item_id'], as_index= False)[['item_cnt_day']].sum()
temp = temp.rename(columns={'item_cnt_day': 'target_by_item_id_total'})
full_df = pd.merge(full_df, temp, on =['date_block_num','item_id'], how= 'left')

temp = sales.groupby(by=['date_block_num','shop_id'], as_index= False)[['item_cnt_day']].sum()
temp = temp.rename(columns={'item_cnt_day': 'target_by_shop_id_total'})
full_df = pd.merge(full_df, temp, on =['date_block_num','shop_id'], how= 'left')

temp = sales.groupby(by=['date_block_num','item_category_id'], as_index= False)[['item_cnt_day']].sum()
temp = temp.rename(columns={'item_cnt_day': 'target_by_category_total'})
full_df = pd.merge(full_df, temp, on =['date_block_num','item_category_id'], how= 'left')

temp = sales.groupby(by=['date_block_num','general_item_category_name'], as_index= False)[['item_cnt_day']].sum()
temp = temp.rename(columns={'item_cnt_day': 'target_by_general_category_total'})
full_df = pd.merge(full_df, temp, on =['date_block_num','general_item_category_name'], how= 'left')

temp = sales.groupby(by=['date_block_num','city'], as_index= False)[['item_cnt_day']].sum()
temp = temp.rename(columns={'item_cnt_day': 'target_by_city_total'})
full_df = pd.merge(full_df, temp, on =['date_block_num','city'], how= 'left')

### Mean

In [ ]:
%%time
temp = sales.groupby(by=['date_block_num','item_id'], as_index= False)[['item_cnt_day']].mean()
temp = temp.rename(columns={'item_cnt_day': 'target_by_item_id_mean'})
full_df = pd.merge(full_df, temp, on =['date_block_num','item_id'], how= 'left')

temp = sales.groupby(by=['date_block_num','shop_id'], as_index= False)[['item_cnt_day']].mean()
temp = temp.rename(columns={'item_cnt_day': 'target_by_shop_id_mean'})
full_df = pd.merge(full_df, temp, on =['date_block_num','shop_id'], how= 'left')

temp = sales.groupby(by=['date_block_num','item_category_id'], as_index= False)[['item_cnt_day']].mean()
temp = temp.rename(columns={'item_cnt_day': 'target_by_category_mean'})
full_df = pd.merge(full_df, temp, on =['date_block_num','item_category_id'], how= 'left')

temp = sales.groupby(by=['date_block_num','general_item_category_name'], as_index= False)[['item_cnt_day']].mean()
temp = temp.rename(columns={'item_cnt_day': 'target_by_general_category_mean'})
full_df = pd.merge(full_df, temp, on =['date_block_num','general_item_category_name'], how= 'left')

temp = sales.groupby(by=['date_block_num','city'], as_index= False)[['item_cnt_day']].mean()
temp = temp.rename(columns={'item_cnt_day': 'target_by_city_mean'})
full_df = pd.merge(full_df, temp, on =['date_block_num','city'], how= 'left')

These statistics helps us to have a full view of target change across different coordinates. Let's take one observation as example. So, we have an item sold in some amount in the current month at some shop, but now we also know, how many of this item we sold in other shops in this month, and how many items of this category we sold, as well as how many those items we sold in this current city. So, now we have more data, even if we look at one raw observation - all data already gathered and reflected in place.

We still have our data marked as data_block_num == 34, what allowed us not to intercept since we've been grouping this by using unique date_block_nums from original dataframe

Let's also add "soft" aggragated data, which includes target data only from past monthes for every "current" month, with the same logic

In [ ]:
full_df.columns

In [ ]:
full_df.head(5)

## First month sold items - { first_month_item_id } 

In [ ]:
full_df['not_full_historical_data'] = 0

We would like to see how many items were sold first time in which monthes:

In [ ]:
first_month = full_df.groupby('item_id', as_index=False)['date_block_num'].min()
first_month['date_block_num'].value_counts().sort_index() # checking - no transformations in this line

As we can see more items started to be sold in the fisrt month, and it's better to assign to these observations "not_full_historical_data" flag positive value. Cause information which we want to extract with this feature - is items and prices outliers. 

In [ ]:
first_month = full_df.groupby('item_id', as_index=False)['date_block_num'].min()
first_month.rename(columns={'date_block_num': 'first_month_item_id_num'}, inplace=True)

full_df = full_df.merge(first_month, on='item_id', how='left')

full_df['first_month_item_id'] = (full_df['date_block_num'] == full_df['first_month_item_id_num']).astype('int8')
full_df = full_df.drop('first_month_item_id_num', axis = 1)
full_df.loc[full_df['date_block_num'] == 0, 'not_full_historical_data'] = 1


In [ ]:
full_df[(full_df['first_month_item_id'] == 1)&(full_df['was_item_cnt_day_outlier'] == 1) ]\
    ['date_block_num'].value_counts().sort_index()

We found clear split, where some items were sold in first month and were "target" (item_cnt_day) outliers as well. Despite the fact 'was_item_price_outlier' in full_df DataFrame is aggregated feature with value only rarelly equal to 1.0 .

In [ ]:
# full_df[full_df['date_block_num'] == 0]

## Expanding window target aggregation

['shop_id','item_id], ['shop_id], ['item_id']

In [ ]:
full_df.head(3)

In [ ]:
aggregating_target_by = [['item_id', 'shop_id'], ['item_id'], ['shop_id']]

In [ ]:
%%time

for feature in aggregating_target_by:
    col = '_'.join(['target_aggregated_mean_premonthes', *feature])
    col2 = '_'.join(['target_aggregated_max_premonthes', *feature])
    full_df[col] = np.nan
    full_df[col2] = np.nan

    for d in notebook.tqdm(full_df.date_block_num.unique()):
        valid_month = (full_df.date_block_num < d)
        current_month = (full_df.date_block_num == d)

        temp = full_df.loc[valid_month].groupby(feature)[['item_cnt_month']].mean().reset_index()
        agg = full_df.loc[current_month][feature].merge(temp, on=feature, how='left')[['item_cnt_month']].copy()
        agg.set_index(full_df.loc[current_month].index, inplace=True)
        full_df.loc[current_month, col] = agg['item_cnt_month']

        temp = full_df.loc[valid_month].groupby(feature)[['item_cnt_month']].max().reset_index()
        agg = full_df.loc[current_month][feature].merge(temp, on=feature, how='left')[['item_cnt_month']].copy()
        agg.set_index(full_df.loc[current_month].index, inplace=True)
        full_df.loc[current_month, col2] = agg['item_cnt_month']

Since this operation takes 6 minutes to load, i will make it easier to debug future steps:

In [ ]:
full_df.to_parquet('full_df.parquet', engine='pyarrow')

## Data Checkpoint

In [ ]:
full_df = pd.read_parquet('full_df.parquet', engine='pyarrow')

### Don't we have some NaNs after target aggreagtion from previous monthes? 

In [ ]:
full_df[full_df['date_block_num'] == 1].columns

Any missed target_aggregated_mean_premonthes_item_id_shop_id from previous monthes?

In [ ]:
full_df[(full_df['target_aggregated_mean_premonthes_item_id_shop_id'].isnull())\
         & (full_df['date_block_num']>0)]\
            [['date_block_num']].value_counts().sort_index()

Let's see without monthes when item was sold first time --> has no data to be aggregated from:

In [ ]:
full_df[(full_df['target_aggregated_mean_premonthes_item_id_shop_id'].isnull())\
         & (full_df['date_block_num']>0) & (full_df['first_month_item_id']!= 1 )]\
            [['date_block_num']].value_counts().sort_index()

In [ ]:
# # In case we can add flag for lack of info for target_aggregated_mean_premonthes_item_id_shop_id with the following code 
# full_df[(full_df['target_aggregated_mean_premonthes_item_id_shop_id'].isnull())\
#          & (full_df['date_block_num']>0) & (full_df['first_month_item_id']!= 1 )]['not_full_historical_data'] = 1

And now let's see if there any shops which have no historical aggregated data:

In [ ]:
# # In case we can add flag for lack of info for shops which were selling first month with the following code 

full_df[(full_df['target_aggregated_mean_premonthes_shop_id'].isnull())\
         & (full_df['date_block_num']>0) & (full_df['first_month_item_id']!= 1 )][['date_block_num']].value_counts().sort_index() # To see what's those NaNs are

# full_df[(full_df['target_aggregated_mean_premonthes_shop_id'].isnull())\
#          & (full_df['date_block_num']>0) & (full_df['first_month_item_id']!= 1 )]['not_full_historical_data'] = 1

After our target aggregation we can see, that we have NaN values, somewehere, because it's first month when item sold, and there no historical data for that item, not possible to collect info about it from previous monthes. We doing well, bacause those observations still have flag for first month selling, they are marked for the model. But what are remaining NaNs, why they still exist if we filter by first month sold items? Those are when item sold not a first time in this month, but sold first time in this shop. Let's check it below only for item_id target value (was it sold prevoisly and has historical data, and was it first time when it sold?):

In [ ]:
full_df[(full_df['target_aggregated_mean_premonthes_item_id'].isnull())\
         & (full_df['date_block_num']>0)]\
            [['date_block_num']].value_counts().sort_index()

In [ ]:
full_df[(full_df['target_aggregated_mean_premonthes_item_id'].isnull())\
         & (full_df['date_block_num']>0) & (full_df['first_month_item_id']!= 1 )]\
            [['date_block_num']].shape

In [ ]:
full_df[full_df['date_block_num']==34].head(5)

### What else? - follow up on the plan

**Basic features:**

- ✅City of the shop  
- ✅Category of the product (item)  
- ✅General product category (item)  
- ✅Monthly sales of exact category in shop `item_cnt_month` (it will be our target) 

**We also should add some lag features based on statistical metrics for exact shops, item_id's, and categories:**

- ✅Total / Mean amount of sold exact `item_id` in this month (How many of this `item_id` we sold in this month?)  
- ✅Total / Mean amount of `item_cnt_day` sold this month in the exact shop (How many items this shop sold this month?)  
- ✅Total / Mean amount of product category sold in this month (How well this category sold during this month?)  
- ✅Total / Mean amount of general product category sold in this month (How well "Movies" category sold during this month?)

**Combined:**
- ❌Mean of how many items of exact category exact shop sell per month (How good this shop sells products of this cateogry?)


**Next:**
- Then we need to add lags for 1, 2, 3, 12 month periods for all features mentioned above  
- ✅Also, would be nice to add a feature like `not_full_historical_data` for the first three months and for monthes in the first year
- Add "deltas" for target - as how amount of sold items been changed for the last monthes of this `shop_id` and `item_id` 
- ✅Add binary feature `first_month_item_id` which reflects if exact item will be sold first time in this month
- ✅❌Add mean sliding window for target for the last three monthes `mean_3`
- Number of month as `month = date_block_num mod 12`
- Add binary feature `shop_was_in_test` and `item_id_was_in_test` for shops and items which will be in test when model will predict

Remaining:
- Number of month as `month = date_block_num mod 12`
- Add binary feature `shop_was_in_test` and `item_id_was_in_test` for shops and items which will be in test when model will predict
- Then we need to add lags for 1, 2, 3, 12 month periods for all features mentioned above  
- Add "deltas" for target - as how amount of sold items been changed for the last monthes of this `shop_id` and `item_id` 

## Year and Month Feature

In [ ]:
full_df['month'] = full_df['date_block_num'] % 12 + 1

In [ ]:
full_df['year'] = 2013 + (full_df['date_block_num'] // 12) 

In [ ]:
full_df['year'].unique()

## Shop_was_in_test and item_id_was_in_test

In [ ]:
shop_id_test = test['shop_id'].unique()
item_id_test = test['item_id'].unique()
print('Nunique shop_id in test: ', test['shop_id'].nunique())
print('Nunique item_id in test: ',test['item_id'].nunique())
print('\n')
print("Unique values of shop_id : ",np.sort(shop_id_test))
print("Unique values of item_id",np.sort(item_id_test))
print('\n')
full_df['item_id_was_in_test'] = 0
full_df['shop_id_was_in_test'] = 0
full_df.loc[(full_df['item_id'].isin(item_id_test))&(full_df['date_block_num'] != 34), 'item_id_was_in_test'] =  1
full_df.loc[full_df['shop_id'].isin(shop_id_test)&(full_df['date_block_num'] != 34), 'shop_id_was_in_test'] =  1

print('Unique values of shop_id with assigned positive shop_id_was_in_test flag',np.sort(full_df[full_df['shop_id_was_in_test']== 1]['shop_id'].unique())) # Checking the result

In [ ]:
size_in_bytes = full_df.memory_usage(deep=True).sum()
size_in_megabytes = size_in_bytes / (1024 ** 2)

print(f"Memory usage full_df: {size_in_megabytes:.2f} MB")
print(f'Amount of rows in this table: {full_df.shape[0]}')
print(f'Amount of columns in this table: {full_df.shape[1]}')

In [ ]:
full_df.head(1)

In [ ]:
# Don't we have any data leakege ? 
full_df.loc[(full_df['date_block_num']== 34)&((full_df['shop_id_was_in_test']!= 0 )|(full_df['item_id_was_in_test']!= 0 )), ['shop_id_was_in_test', 'item_id_was_in_test']].shape[0]

## Downcast

In [ ]:
full_df.info()

In [ ]:
float_cols = full_df.select_dtypes(include='float64').columns
int_cols = full_df.select_dtypes(include='int64').columns

for col in float_cols:
    max_diff = (full_df[col] - full_df[col].astype('float32')).abs().max()
    print(f"{col}: max precision loss when downcasted to float32 = {max_diff}")

for col in int_cols:
    min_val = full_df[col].min()
    max_val = full_df[col].max()
    if min_val < -2_147_483_648 or max_val > 2_147_483_647:
        print(f"{col}: OVERFLOW when downcasted to int32 (values out of range)")


In [ ]:
def downcast_dtypes(df):
    float_cols = df.select_dtypes(include=['float64']).columns
    int_cols = df.select_dtypes(include=['int64']).columns

    df[float_cols] = df[float_cols].astype('float32')
    df[int_cols] = df[int_cols].astype('int32')

    return df

In [ ]:
size_in_bytes = full_df.memory_usage(deep=True).sum()
size_in_megabytes = size_in_bytes / (1024 ** 2)

print(f"Memory usage full_df: {size_in_megabytes:.2f} MB")
print(f'Amount of rows in this table: {full_df.shape[0]}')
print(f'Amount of columns in this table: {full_df.shape[1]}')

In [ ]:
full_df.isna().sum()

item_cnt_month aggregation by shop, item_id etc. are expectably NaNs for test, for other features we can fill with 0 valu, since amount of sold items in previous monthes is zero

In [ ]:
test.shape[0] 

In [ ]:
%%time
full_df.fillna(0, inplace=True)
full_df = downcast_dtypes(full_df)

In [ ]:
size_in_bytes = full_df.memory_usage(deep=True).sum()
size_in_megabytes = size_in_bytes / (1024 ** 2)

print(f"Memory usage full_df: {size_in_megabytes:.2f} MB")
print(f'Amount of rows in this table: {full_df.shape[0]}')
print(f'Amount of columns in this table: {full_df.shape[1]}')

In [ ]:
full_df.info()

## Lag features

In [ ]:
all_obs_combination_by = ['date_block_num', 'shop_id', 'item_id']

In [ ]:
shifted_columns = [c for c in full_df if 'target' in c]
shifted_columns

In [ ]:
%%time
all_obs_combination_by = ['date_block_num', 'shop_id', 'item_id']

shift_range = [1, 2, 3, 12]

shifted_columns = [c for c in full_df if 'target' in c]

for shift in shift_range:
    temp = full_df[all_obs_combination_by + shifted_columns].copy()
    temp['date_block_num'] = temp['date_block_num'] + shift

    foo = lambda x: f'{x}_lag_{shift}' if x in shifted_columns else x
    temp = temp.rename(columns=foo)

    full_df = pd.merge(full_df, temp, on = all_obs_combination_by, how= 'left').fillna(0)
    full_df = downcast_dtypes(full_df)

    del temp
    gc.collect

In [ ]:
full_df.shape

In [ ]:
full_df.tail(5)

In [ ]:
np.array(full_df.columns)

In [ ]:
size_in_bytes = full_df.memory_usage(deep=True).sum()
size_in_megabytes = size_in_bytes / (1024 ** 2)

print(f"Memory usage full_df: {size_in_megabytes:.2f} MB")
print(f'Amount of rows in this table: {full_df.shape[0]}')
print(f'Amount of columns in this table: {full_df.shape[1]}')

### Deltas